# Data Cleaning with Pandas

An important step in the process of working with data is cleaning. In the real world, data is often very messy. As a data engineer, you will find yourself in the position of working with data that is coming in from multiple sources, and might not all be in a single standardized format. Your data might have a lot of missing values, or inconsistent values within a single column, or misplaced values, or any number of other issues. Poor quality data is hard to work with, reduces the reliability of any findings which might be based on it, and could raise regulatory and other issues. In short, there are many reasons to make your data as clean as possible before beginning to work with it.

Often when you download a dataset from a data science or other website, the data has already been cleaned up. That obviously makes the data easier to work with, but it can be very misleading, because it overlooks an important part of the process that someone is responsible for. As a data engineer, you will often be the one responsible for cleaning the data.

Let's take a look at one of our datasets to see the cleaning process in action. We'll look at the [aircraft data from chapter 3](../../../data/input/ch3), because it is a large dataset with a lot of columns.

First, let's do our standard imports and data loading:

In [2]:
import pandas as pd

In [17]:
data_dir = '../../../data/input/ch3/'
aircraft_file = data_dir + 'deb-aircrafts.csv'
aircraft = pd.read_csv(aircraft_file, header=0)

In [3]:
print(aircraft.head())
print(aircraft.shape)

  n_number serial_number mfr_mdl_code  eng_mfr_mdl  mfr_year registrant_type  \
0     N100          5334      7100510      17003.0      1940      Individual   
1   N10001           A28      9601202      67007.0      1928      Individual   
2   N10002        79-030      8930105      41525.0      1979     Corporation   
3   N10003             1      056336T          NaN        -1      Individual   
4   N10004     T18208245      2072738          NaN        -1             LLC   

       registrant_name            street street2         city  ...  \
0                  NaN               NaN     NaN      KETCHUM  ...   
1                  NaN               NaN     NaN     LAKELAND  ...   
2  ENGLISH AIR SERVICE  3409 CORSAIR CIR     NaN  SANTA MARIA  ...   
3                  NaN               NaN     NaN    SALISBURY  ...   
4         ETOS AIR LLC               NaN     NaN   NEW LONDON  ...   

              aircraft_type num_engines num_seats weight_class speed  \
0  Fixed wing single engin

At first glance, we can see that the `street` and `street2` columns have a lot of empty values. Since there are so many missing values, and these columns are probably of ancillary importance to someone who is using the data to make a business decision, let's remove those columns from our `DataFrame`. Pandas makes this easy with the `drop()` function:

In [18]:
# prepare the list of columns to be dropped
drop_cols = ['street','street2']
aircraft.drop(drop_cols, inplace=True, axis=1)

# print the updated DataFrame
print(aircraft.head())

  n_number serial_number mfr_mdl_code  eng_mfr_mdl  mfr_year registrant_type  \
0     N100          5334      7100510      17003.0      1940      Individual   
1   N10001           A28      9601202      67007.0      1928      Individual   
2   N10002        79-030      8930105      41525.0      1979     Corporation   
3   N10003             1      056336T          NaN        -1      Individual   
4   N10004     T18208245      2072738          NaN        -1             LLC   

       registrant_name         city state zip_code  ...  \
0                  NaN      KETCHUM    OK    74349  ...   
1                  NaN     LAKELAND    FL    33802  ...   
2  ENGLISH AIR SERVICE  SANTA MARIA    CA    93455  ...   
3                  NaN    SALISBURY    NC    28146  ...   
4         ETOS AIR LLC   NEW LONDON    TX    75682  ...   

              aircraft_type num_engines num_seats weight_class speed  \
0  Fixed wing single engine           1         2      CLASS 1    67   
1  Fixed wing single

Now we can see that the `street` and `street2` columns have been removed from our `aircraft` DataFrame. 

Another issue that we can see with our data is that the `thrust` column is also somewhat messy. Some of the entries are empty, some contain zeros, and some contain meaningful values. Let's replace the `NaN`s with zeros, using the `fillna()` function, so that the unrecorded values are consistent:

In [13]:
# replace NaNs with 0s using fillna()
aircraft['thrust'] = aircraft['thrust'].fillna(0)

print(aircraft.head())

  n_number serial_number mfr_mdl_code  eng_mfr_mdl  mfr_year registrant_type  \
0     N100          5334      7100510      17003.0      1940      Individual   
1   N10001           A28      9601202      67007.0      1928      Individual   
2   N10002        79-030      8930105      41525.0      1979     Corporation   
3   N10003             1      056336T          NaN        -1      Individual   
4   N10004     T18208245      2072738          NaN        -1             LLC   

       registrant_name         city state zip_code  ...  \
0                  NaN      KETCHUM    OK    74349  ...   
1                  NaN     LAKELAND    FL    33802  ...   
2  ENGLISH AIR SERVICE  SANTA MARIA    CA    93455  ...   
3                  NaN    SALISBURY    NC    28146  ...   
4         ETOS AIR LLC   NEW LONDON    TX    75682  ...   

              aircraft_type num_engines num_seats weight_class speed  \
0  Fixed wing single engine           1         2      CLASS 1    67   
1  Fixed wing single

Another common cleaning operation is to drop duplicate rows. If we want to see whether any rows are duplicated, we can use the `duplicated()` method on our DataFrame. In this case, there are no duplicate entries. If you do need to remove duplicates, Pandas provides a handy `drop_duplicates()` function. 

In [32]:
print(aircraft.duplicated())

#if you need to remove duplicates, use drop_duplicates()
aircraft.drop_duplicates()

n_number
N100      False
N10001    False
N10002    False
N10003    False
N10004    False
          ...  
N9ZS      False
N9ZT      False
N9ZU      False
N9ZV      False
N9ZX      False
Length: 288116, dtype: bool
         serial_number mfr_mdl_code  eng_mfr_mdl  mfr_year registrant_type  \
n_number                                                                     
N100              5334      7100510      17003.0      1940      Individual   
N10001             A28      9601202      67007.0      1928      Individual   
N10002          79-030      8930105      41525.0      1979     Corporation   
N10003               1      056336T          NaN        -1      Individual   
N10004       T18208245      2072738          NaN        -1             LLC   
...                ...          ...          ...       ...             ...   
N9ZS             20004      5760102          0.0      1974      Individual   
N9ZT              0088      2130001      17042.0      2001     Corporation   
N9ZU   

Notice from the output and row counts above that when using the default behavior of `drop_duplicates()` (which is to compare across all columns), there are no duplicate rows in our data. While there may not be any duplicates when we compare records according to all of the columns, `drop_duplicates()` gives us a way to filter out duplicates according to more detailed criteria. In particular, we can pass a list of columns using the `subset` argument, and `drop_duplicates()` will check for duplicates only on those specific columns. 

Let's suppose that, for some regulatory purpose, we need to produce a recordset containing one aircraft from every airline in each ZIP code, and that only contains corporate (and not individual) registrants. We can do this easily with `drop_duplicates` using `subset`. 

First, though, we need to use `dropna()` to drop records with a `NaN` value for the `registry_name`. This will leave only corporate registrants. Then, we can apply `drop_duplicates()` specifying `registrant_name` and `zip_code` as the subset.

In [19]:
# drop individual registrants
aircraft.dropna(subset=['registrant_name'], inplace=True)

# remove duplicate registrants within each ZIP code
print(aircraft.drop_duplicates(subset=['registrant_name', 'zip_code'], keep='last').iloc[: , :15].head(10))

   n_number  serial_number mfr_mdl_code  eng_mfr_mdl  mfr_year  \
9    N1000E       550-1011      2076604      52065.0      2002   
16   N1000Q       310P0195      2074238      17027.0        -1   
20   N1000V           0660      2130001      17042.0      2003   
21   N1000W  CCX-2300-0017      05639MP          NaN        -1   
33   N1001D           7609      2073110      35003.0      1950   
35   N1001F          79033      8930105      42200.0      1979   
37   N1001K     F-16-07-01      060005B      55593.0      2017   
44   N1001W          60208      05620P0      41594.0      2006   
49   N10025            165      8141104      52008.0      1957   
54   N1002G        27-0295      5870222      41533.0      2000   

   registrant_type          registrant_name           city state zip_code  \
9      Corporation               WR HESS CO  OKLAHOMA CITY    OK    73119   
16             LLC               N1000Q LLC        OAKLAND    CA    94605   
20             LLC                  1KV LL

Notice, in the last line, that we can append multiple function calls to a DataFrame reference to handle more than one processing step in the same statement. `drop_duplicates()` will return a DataFrame, `iloc` uses indexing to return only the first 15 columns (so the result is easier to see), and `head()` shows us only the first few rows.

Every dataset is different, and the exact procedure for cleaning your data will depend on what you find when you perform your initial exploratory analysis. Cleaning data will make your job, and that of the data scientists and analysts, easier, and add value to the process. 

### Further Reading
- https://realpython.com/python-data-cleaning-numpy-pandas/